In [62]:
!pip install tqdm

In [5]:
from pprint import pprint
from Parser import Parser
import util
from tfidf import *
import glob, os
import math
import numpy as np
from tqdm import tqdm
from __future__ import division, unicode_literals
from textblob import TextBlob as tb

class VectorSpace:
    """ A algebraic model for representing text documents as vectors of identifiers. 
    A document is represented as a vector. Each dimension of the vector corresponds to a 
    separate term. If a term occurs in the document, then the value in the vector is non-zero.
    """

    #Collection of document term vectors
    documentVectors = []

    #Mapping of vector index to keyword
    vectorKeywordIndex=[]

    #Tidies terms
    parser=None
    
    def __init__(self, documents=[], vectorMode = 'tf'):
        self.documentVectors=[]
        self.parser = Parser()
        self.BlobList = self.getBlobList(documents)
        self.vectorMode = vectorMode
        if(len(documents)>0):
            self.build(documents)
    
    def getBlobList(self, documents): 
        bloblist = []
        for doc in documents:
            wordList = self.parser.tokenise(doc)
            wordList = self.parser.removeStopWords(wordList)
            bloblist.append(tb(" ".join(wordList)))
        return bloblist
            
    def build(self,documents):
        """ Create the vector space for the passed document strings """
        self.vectorKeywordIndex = self.getVectorKeywordIndex(documents)
        self.documentVectors = [self.makeVector(document, self.vectorMode) for document in tqdm(documents)]

        #print(self.vectorKeywordIndex)
        #print(self.documentVectors)


    def getVectorKeywordIndex(self, documentList):
        """ create the keyword associated to the position of the elements within the document vectors """

        #Mapped documents into a single word string	
        vocabularyString = " ".join(documentList)

        vocabularyList = self.parser.tokenise(vocabularyString)
        #print(vocabularyList)
        #print(vocabularyString)
        #Remove common words which have no search value
        vocabularyList = self.parser.removeStopWords(vocabularyList)
        #print(vocabularyList)
        uniqueVocabularyList = util.removeDuplicates(vocabularyList)
        #print(uniqueVocabularyList)
        vectorIndex={}
        offset=0
        #Associate a position with the keywords which maps to the dimension on the vector used to represent this word
        for word in uniqueVocabularyList:
            vectorIndex[word]=offset
            offset+=1
        return vectorIndex  #(keyword:position)


    def makeVector(self, wordString, mode):
        """ @pre: unique(vectorIndex) """
        #Initialise vector with 0's
        vector = [0] * len(self.vectorKeywordIndex)
        wordList = self.parser.tokenise(wordString)
        wordList = self.parser.removeStopWords(wordList)
        tbString = tb(" ".join(wordList))
        if mode == 'tf':
            for word in list(set(wordList)):
                vector[self.vectorKeywordIndex[word]] = tf(word, tbString) #Use simple Term Count Model
            return vector 
        
        if mode == 'tf-idf':
            #print('bloblist:', self.BlobList)
            for word in list(set(wordList)):
                print('word',word)
                vector[self.vectorKeywordIndex[word]] =  tfidf(word, tbString , self.BlobList) 
                #print('word:',word, 'idf:', idf(word, self.BlobList),  )
                #print('word:', word, 'tf:', tf(word, tbString))
            return vector
            
    '''
    def tf(self, word, blob):
        return blob.words.count(word) / len(blob.words)
    
    def n_containing(self, word, bloblist):
        print()
        return sum(1 for blob in bloblist if word in blob.words)

    def idf(self, word, bloblist):
        print('總文件數量：', len(bloblist), '含有{}字的文章數量+1:'.format(word), (1 + self.n_containing(word, bloblist)))
        print('{}的idf為:'.format(word), math.log(len(bloblist) / (1 + self.n_containing(word, bloblist))))
        return math.log(len(bloblist) / (1 + self.n_containing(word, bloblist)))

    def tfidf(self, word, blob, bloblist):
        return tf(word, blob) * idf(word, bloblist)
    '''
    
    def old_makeVector(self, wordString):
        """ @pre: unique(vectorIndex) """

        #Initialise vector with 0's
        vector = [0] * len(self.vectorKeywordIndex)
        wordList = self.parser.tokenise(wordString)
        wordList = self.parser.removeStopWords(wordList)
        for word in wordList:
            vector[self.vectorKeywordIndex[word]] += 1; #Use simple Term Count Model
        return vector


    def buildQueryVector(self, termList):
        """ convert query string into a term vector """
        #print(termList)
        #print(self.vectorMode)
        query = self.makeVector(" ".join(termList), self.vectorMode)
        return query


    def related(self,documentId):
        """ find documents that are related to the document indexed by passed Id within the document Vectors"""
        ratings = [util.cosine(self.documentVectors[documentId], documentVector) for documentVector in self.documentVectors]
        #ratings.sort(reverse=True)
        return ratings
    
    def search(self,searchList, mode = 'cos'):
        """ search for documents that match based on a list of terms """
        #print(searchList)
        queryVector = self.buildQueryVector(searchList)
        #print(queryVector)
        if mode == 'cos':
            ratings = [util.cosine(queryVector, documentVector) for documentVector in tqdm(self.documentVectors)]
        #ratings.sort(reverse=True)
            return ratings
        if mode == 'eucli':
            ratings = [util.Euclidean(queryVector, documentVector) for documentVector in tqdm(self.documentVectors)]
            return ratings

    def old_search(self,searchList):
        """ search for documents that match based on a list of terms """
        queryVector = self.buildQueryVector(searchList)
        #print(queryVector)
        ratings = [util.cosine(queryVector, documentVector) for documentVector in self.documentVectors]
        #ratings.sort(reverse=True)
        return ratings
    
    def printresult(self,searchlist, mode='cos'):
        scoreList = self.search(searchlist, mode = mode)
        for i in np.flip(np.argsort(scoreList))[:10]:
            print('score:', scoreList[i], 'document:' , documents[i])

In [53]:
#test data

documents = ["The cat cat in the hat disabled",
                 "A cat is a fine pet ponies.",
                 "Dogs and cats make good pets.",
                 "I haven't got a hat."]

vectorSpace = VectorSpace(documents, 'tf-idf')  # vectorSpace(documents, vectorMode = 'tf' or 'tf-idf') (default is 'tf')


#print(vectorSpace.vectorKeywordIndex)

#print(vectorSpace.documentVectors)

#print(vectorSpace.related(1))

#print(vectorSpace.search(["cat"]))   

vectorSpace.printresult(["cat"])
vectorSpace.printresult(["cat"], mode = 'eucli')  # mode = 'cos' or 'eucli' (default is 'cos')

###################################################

score: nan document: I haven't got a hat.
score: nan document: Dogs and cats make good pets.
score: nan document: A cat is a fine pet ponies.
score: nan document: The cat cat in the hat disabled
score: 1.2345525572306575 document: Dogs and cats make good pets.
score: 1.0216002166437488 document: A cat is a fine pet ponies.
score: 0.7504758415354574 document: The cat cat in the hat disabled
score: 0.28768207245178085 document: I haven't got a hat.


In [14]:
import numpy as np
scoreList = vectorSpace.search(["cat"], mode = 'cos')
for i in np.flip(np.argsort(scoreList)):
    print('score:', scoreList[i], 'document:' , documents[i])
'''
for score in vectorSpace.search(["cat"]):
    print('score:', score)
    print(np.argsort(vectorSpace.search(["cat"])))
'''

[0, 0, 0, 0, 0, 0, 1.0, 0, 0]
score: 0.8164965809277261 document: The cat cat in the hat disabled
score: 0.5 document: A cat is a fine pet ponies.
score: 0.4472135954999579 document: Dogs and cats make good pets.
score: 0.0 document: I haven't got a hat.


'\nfor score in vectorSpace.search(["cat"]):\n    print(\'score:\', score)\n    print(np.argsort(vectorSpace.search(["cat"])))\n'

In [22]:
'''
# try euclidean distance
# euclidean distance = ((x1 - y1)^2 + (x2 - y2)^2) ^(1/2)
from numpy.linalg import norm
import numpy as np
x = [1, 2, 3]
y = [4, 5, 6]
array1 = np.array(x)
array2 = np.array(y)
subtracted_array = np.subtract(array1, array2)
subtracted = list(subtracted_array)
eucli = norm(subtracted)
print(eucli)
np.sum((array1 - array2)**2)**0.5
'''

5.196152422706632


5.196152422706632

In [3]:
'''
with open("./EnglishNews/EnglishNews/News100012.txt", encoding = 'utf-8') as f:
    lines = f.readlines()
    doc = ' '.join(lines)
    doc1 = doc.replace("\n", "")

print(doc1)
'''

documents = []
bloblist = []
for file in os.listdir("./EnglishNews/EnglishNews"):
    if file.endswith(".txt"):
        filename = os.path.join("./EnglishNews/EnglishNews", file)
        bloblist.append(filename)
        with open(filename, encoding="utf-8") as f:
            lines = f.readlines()
            doc = ' '.join(lines)
            doc1 = doc.replace("\n", "")
            documents.append(doc1)

In [59]:
documents

["Trump aide who stormed the Capitol broke an 'oath to protect America,' judge says WASHINGTON (Reuters) - A man appointed to the U.S. State Department during the Trump administration will remain in jail while he awaits trial on charges that he took part in the deadly storming of the U.S. Capitol and assaulted police officers, a judge said on Tuesday.",
 "U.N. Security Council condemns Myanmar violence, U.S. blacklists army leader's children (Reuters) - The U.N. Security Council on Wednesday condemned violence against Myanmar protesters and called on the army to show restraint, but failed to denounce the military takeover as a coup or threaten further action due to opposition from China and Russia.",
 "Lobbyist to be paid $2 million to 'explain' Myanmar's coup on behalf of junta WASHINGTON (Reuters) - An Israeli-Canadian lobbyist hired by Myanmar’s junta will be paid $2 million to “assist in explaining the real situation” of the army’s coup to the United States and other countries, doc

In [68]:
vectorSpace_tfidf = VectorSpace(documents, 'tf-idf')
vectorSpace_tfidf.printresult(["Trump Biden Taiwan China"])
vectorSpace_tfidf.printresult(["Trump Biden Taiwan China"], mode = 'eucli')

100%|██████████████████████████████████████████████████████████████████████████████| 7034/7034 [31:52<00:00,  3.68it/s]


KeyError: 'taiwan'

In [6]:
vectorSpace_tfidf_500 = VectorSpace(documents[:500], 'tf-idf')
vectorSpace_tfidf_500.printresult(["Trump Biden Taiwan China"])

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]

word administr


  1%|▋                                                                                 | 4/500 [00:00<00:52,  9.44it/s]

word depart
word (reuters)
word jail
word part
word storm
word -
word judg
word aid
word tuesdai
word protect
word america,'
word polic
word appoint
word await
word trump
word man
word assault
word deadli
word washington
word dure
word remain
word capitol
word officers,
word trial
word sai
word state
word 'oath
word charg
word broke
word children
word violence,
word call
word (reuters)
word russia
word china
word denounc
word -
word due
word myanmar
word threaten
word wednesdai
word condemn
word opposit
word protest
word takeov
word restraint,
word coup
word armi
word violenc
word leader'
word secur
word blacklist
word show
word fail
word action
word militari
word council
word (reuters)
word file
word israeli-canadian
word explain
word -
word hire
word million
word junta
word govern
word $2
word coup
word countries,
word unit
word washington
word army’
word document
word real
word show
word paid
word myanmar'
word lobbyist
word 'explain'
word state
word behalf
word situation”
word “ass

  2%|█▊                                                                               | 11/500 [00:00<00:25, 19.14it/s]

...

 britain’
word world
word moore,
word social
word christian
word (csu)
word (reuters)
word tricki
word wednesday,
word polici
word partner
word -
word smaller
word conserv
word alliance,
word cdu/csu
word union
word coalit
word green
word bavarian
word greens’
word leader
word current
word german
word liber
word berlin
word ad
word platform
word suitabl
word prospect,
word sai
word (reuters)
word minist
word eu'
word border
word countri
word presid
word prime
word andrej
word -
word impos
word michel
word eas
word wednesdai
word good
word czech
word germani
word industri
word charl
word central
word babi
word european
word components,
word german
word control
word pragu
word austrian
word pm
word free
word truck
word flow
word council
word (reuters)
word presid
word post-trump
word -
word move
word inclin
word leav
word hope
word express
word wednesdai
word legisl
word offic
word covid-19
word januari
word republican
word trump
word forward
word president-elect
word washington
word don

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:19<00:00, 25.15it/s]



word alreadi
word infect
word antibodies,
word passed,
word sai
word rate
word amid
word concern
word (reuters)
word onlin
word china-styl
word ha
word threat
word democrat
word -
word monitored,
word crackdown
word gatewai
word opposit
word govern
word freedom
word penh
word establish
word local
word decre
word issu
word phnom
word adopt
word internet
word control
word prompt
word cambodia
word cambodia’
word traffic
word taiwan
word trump
word china
word biden


100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:03<00:00, 154.28it/s]

score: 0.39136102641481313 document: Taiwan says international participation discussed in call with U.S. envoy TAIPEI (Reuters) - Taiwan’s presidential office said on Thursday that President Tsai Ing-wen and the U.S. ambassador to the United Nations Kelly Craft had discussed Taiwan’s international participation and deepening Taiwan-U.S. ties during a call between the two.
score: 0.30805188488657254 document: U.S., Canada hail Taiwan's 'freedom pineapples' after Chinese ban TAIPEI (Reuters) - The de facto U.S. and Canadian embassies in Taiwan on Tuesday praised the quality of pineapples grown on the island, depicting photographs of their top diplomats in Taipei with the fruit after an import ban by China.
score: 0.290510077049025 document: China says U.S. undermining stability after U.S. warship sails through Taiwan Strait BEIJING (Reuters) - The Chinese military criticised the United States on Thursday for undermining regional peace and stability after a U.S. Navy warship sailed throug

In [85]:
print(vectorSpace_tfidf_500.vectorKeywordIndex)

{'draw': 0, 'new,': 1, 'pan': 2, 'beijing’': 3, 'barcelona': 4, 'blend': 5, '(miles)': 6, 'bans,': 7, 'korea': 8, 'pose': 9, 'group:': 10, 'strategi': 11, 'relect': 12, 'iraq,': 13, 'teams,': 14, 'direct': 15, 'regul': 16, 'cooper': 17, 'ngo': 18, 'buri': 19, 'nemzet': 20, 'protests,': 21, 'improv': 22, 'opportun': 23, 'vajiralongkorn': 24, '79;': 25, 'reviv': 26, 'austin': 27, 'shift': 28, 'economy,': 29, 'us-mexico': 30, 'payment': 31, 'pharmaceut': 32, 'interfax': 33, 'haiti': 34, 'tigrai': 35, 'touch': 36, 'recreat': 37, "president'": 38, 'thierri': 39, 'stake': 40, 'electron': 41, 'long': 42, 'ismail': 43, 'bar': 44, 'reduc': 45, 'vertigin': 46, 'respect': 47, 'weaken': 48, 'rabat': 49, 'long-gon': 50, 'moldova': 51, 'logist': 52, '“el': 53, 'intellig': 54, 'iraqi': 55, 'chapo”': 56, 'patrol': 57, 'intimid': 58, 'reach': 59, 'pfizer’': 60, 'popular': 61, 'markl': 62, 'year': 63, 'repres': 64, 'onli': 65, 'withstand': 66, 'cup': 67, 'defus': 68, 'well-known': 69, '50-year': 70, 'mo

In [91]:
print(vectorSpace_tfidf_500.vectorKeywordIndex['taiwan'])
print(vectorSpace_tfidf_500.vectorKeywordIndex['taiwan'])

1284
1284


In [70]:
print(vectorSpace_tfidf_100.vectorKeywordIndex)

{'new,': 0, 'korea': 1, 'strategi': 2, 'regul': 3, 'improv': 4, 'economy,': 5, 'pharmaceut': 6, 'touch': 7, 'thierri': 8, 'long': 9, 'respect': 10, 'weaken': 11, 'intellig': 12, 'iraqi': 13, 'reach': 14, 'pfizer’': 15, 'markl': 16, 'year': 17, 'repres': 18, 'onli': 19, 'well-known': 20, 'lobbyist': 21, 'acr': 22, 'texas,': 23, 'york,': 24, 'breton': 25, 'watchdog': 26, 'panhandl': 27, 'divorc': 28, 'saturday,': 29, "us'": 30, 'xi': 31, 'people,': 32, 'confirm': 33, "eu'": 34, 'parties”': 35, 'latest': 36, 'show': 37, 'forc': 38, 'donald': 39, 'sd': 40, 'wa': 41, 'protest': 42, '30': 43, 'achiev': 44, 'judg': 45, 'capit': 46, 'interview': 47, 'pursuit': 48, 'sundai': 49, 'duterte’': 50, 'congress': 51, '(who)': 52, 'afghanistan': 53, 'pm': 54, 'indian': 55, 'similar': 56, 'parti': 57, 'prigozhin,': 58, 'assault': 59, 'uk': 60, 'farmer': 61, 'hair': 62, 'sein': 63, 'chair': 64, 'raphael': 65, 'eas': 66, 'scrutini': 67, 'arrangement,': 68, '“genuin': 69, 'citizen': 70, 'joint': 71, 'dai':

In [77]:
parser = Parser()
string = "Taiwan tensions don't need to lead to 'anything like confrontation': Pentagon WASHINGTON (Reuters) - The Pentagon reaffirmed its longstanding support to Taiwan’s self-defense on Thursday and said there was no reason that tensions between China and Taiwan “need to lead to anything like confrontation.”"
tokens = parser.tokenise(string)
tokens = parser.removeStopWords(tokens)
uniqueTokens = util.removeDuplicates(tokens)
vectorIndex={}
offset=0
        #Associate a position with the keywords which maps to the dimension on the vector used to represent this word
for word in uniqueTokens:
    vectorIndex[word]=offset
    offset+=1
print(vectorIndex)

{'washington': 0, 'reaffirm': 1, 'longstand': 2, 'taiwan': 3, '-': 4, "confrontation':": 5, 'reason': 6, "'anyth": 7, 'tension': 8, 'thursdai': 9, '(reuters)': 10, '“need': 11, 'self-defens': 12, 'china': 13, 'taiwan’': 14, 'confrontation”': 15, 'anyth': 16, 'wa': 17, 'support': 18, 'lead': 19, 'pentagon': 20}


In [ ]:
vectorSpace_tf = VectorSpace(documents, 'tf')
vectorSpace_tf.printresult(["Trump Biden Taiwan China"])
vectorSpace_tf.printresult(["Trump Biden Taiwan China"], mode = 'eucli')

In [126]:
'''
from __future__ import division, unicode_literals
import math
from textblob import TextBlob as tb

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    
    return tf(word, blob) * idf(word, bloblist)

document1 = tb("""Python is a 2000 made-for-TV horror movie directed by Richard Clabaugh. The film features several cult favorite actors, including William Zabka of The Karate Kid fame, Wil Wheaton, Casper Van Dien, Jenny McCarthy, Keith Coogan, Robert Englund (best known for his role as Freddy Krueger in the A Nightmare on Elm Street series of films), Dana Barron, David Bowe, and Sean Whalen. The film concerns a genetically engineered snake, a python, that escapes and unleashes itself on a small town. It includes the classic final girl scenario evident in films like Friday the 13th. It was filmed in Los Angeles, California and Malibu, California. Python was followed by two sequels: Python II (2002) and Boa vs. Python (2004), both also made-for-TV films.""")
document2 = tb("""Python, from the Greek word, is a genus of nonvenomous pythons[2] found in Africa and Asia. Currently, 7 species are recognised.[2] A member of this genus, P. reticulatus, is among the longest snakes known.""")
document3 = tb("""The Colt Python is a .357 Magnum caliber revolver formerly manufactured by Colt's Manufacturing Company of Hartford, Connecticut.  It is sometimes referred to as a "Combat Magnum".[1] It was first introduced in 1955, the same year as Smith &amp; Wesson's M29 .44 Magnum. The now discontinued Colt Python targeted the premium revolver market segment. Some firearm collectors and writers such as Jeff Cooper, Ian V. Hogg, Chuck Hawks, Leroy Thompson, Renee Smeets and Martin Dougherty have described the Python as the finest production revolver ever made.""")


bloblist = [document1, document2, document3]

for i, blob in enumerate(bloblist):
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words} # 計算該文字的tf-idf值
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:3]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

print(scores)
'''

Top words in document 1
	Word: python, TF-IDF: 0.01662
	Word: films, TF-IDF: 0.00997
	Word: made-for-TV, TF-IDF: 0.00665
Top words in document 2
	Word: genus, TF-IDF: 0.02253
	Word: 2, TF-IDF: 0.02253
	Word: from, TF-IDF: 0.01126
Top words in document 3
	Word: Colt, TF-IDF: 0.01367
	Word: Magnum, TF-IDF: 0.01367
	Word: revolver, TF-IDF: 0.01367
{'The': 0.0, 'Colt': 0.013667363194657226, 'Python': -0.009697148509610592, 'is': -0.006464765673073728, 'a': -0.006464765673073728, '357': 0.0045557877315524084, 'Magnum': 0.013667363194657226, 'caliber': 0.0045557877315524084, 'revolver': 0.013667363194657226, 'formerly': 0.0045557877315524084, 'manufactured': 0.0045557877315524084, 'by': 0.0, "'s": 0.009111575463104817, 'Manufacturing': 0.0045557877315524084, 'Company': 0.0045557877315524084, 'of': -0.003232382836536864, 'Hartford': 0.0045557877315524084, 'Connecticut': 0.0045557877315524084, 'It': 0.0, 'sometimes': 0.0045557877315524084, 'referred': 0.0045557877315524084, 'to': 0.00455578773